In [85]:
import pandas as pd
import numpy as np
import multiprocessing as mp

import pickle
import json
import os
from tqdm import tqdm

from collections import Counter, OrderedDict

In [2]:
data_folder = '../../data'
def get_file(path):
    return os.path.join(data_folder, path)

In [3]:
index = pd.read_csv(get_file('index.csv'))
index.head()

,participant_id,handle,contest_id,problem_lit,problem_id,solution_id,path
0,4103956,teja.gravi,1,A,1,10005159,codeforces\1\A\submissions\10005159.program.cpp
1,4103956,teja.gravi,1,A,1,10005235,codeforces\1\A\submissions\10005235.program.cpp
2,2911164,3abdo,1,A,1,10007229,codeforces\1\A\submissions\10007229.program.cpp
3,4117284,Ybitsa2002,1,A,1,10007877,codeforces\1\A\submissions\10007877.program.cpp
4,4117209,kazisami,1,A,1,10010032,codeforces\1\A\submissions\10010032.program.cpp


In [4]:
index.shape

(13876756, 7)

### Remove submits from other online judges

In [5]:
handle_counts = index.handle.value_counts()
handles, handle_counts = handle_counts.index, handle_counts.values

In [6]:
for handle, cnt in zip(handles, handle_counts):
    if 'judge' in handle:
        print(handle, cnt)

vjudge4 215306
vjudge5 215034
vjudge3 214359
vjudge1 214286
vjudge2 213752
bnuvjudge 6502
bnuvjudge2 3811
bnuvjudge3 1781
judge 976
vjudge999 650
ntu_vjudge_1 486
sdauvjudge 469
ntu_vjudge_2 459
anothervjudge 365
nitvjudge1 343
nitvjudge2 316
wust_judge_4 268
CSUvjudge1 260
wust_judge_2 253
CSUvjudge2 249
wust_judge_3 248
wust_judge_1 237
CSUvjudge3 220
hdujudge5 194
hdujudge1 188
hdujudge9 188
hdujudge8 187
chdvjudge 186
hdujudge7 182
hdujudge10 179
hdujudge3 179
hdujudge4 170
ntu-vjudge 168
hdujudge2 164
judgee 161
hdujudge6 159
wust_judge_5 154
nitvjudge3 144
vjudge18 112
nitvjudge4 103
nsoi_vjudge 94
nsoi_vjudge2 92
zjnuvjudge1 83
zjnuvjudge2 81
judgeee 80
hpuvjudge 74
zjnuvjudge3 74
vjudge7 70
AdzearDisjudge 67
nswojvjudge3 67
rejudgeX 60
nsoi_vjudge1 55
vjudge77 55
zjudge 53
judgejltiet2 51
vjudge8 50
codejudge 50
vjudge91 48
zjnuvjudge5 47
judgeopi 41
sjudge2 39
vjudge22 34
vjudge88 33
judgeeeeee 30
sjudge1 29
sjudge3 28
vjudge108 25
vjudge99 24
vjudge23 21
vjudge21 18
vjudge735

In [7]:
def invalid(handle):
    handle = str(handle)
    bad_guys = ['Scut82', 'timeufpe', 'maratonando', 'maratonando2', 'maratonando3', 'Bannedaccount']
    return 'luogu_bot' in handle.lower() or 'judge' in handle.lower() or handle in bad_guys

judge_locations = np.array([invalid(handle) for handle in index.handle])

In [8]:
index = index.loc[~judge_locations]

In [9]:
index.shape

(12665495, 7)

In [10]:
handle_counts = index.handle.value_counts()

### Now top-submitted users are real, with orange to red rating

In [11]:
handle_counts[:20]

SoiMae                   4057
guille                   3772
kmjp                     3395
dreamoon_love_AA         3313
TripleM5da               3174
krijgertje               3159
KrK                      3126
I_love_Tanya_Romanova    3060
Dalgerok                 2855
ivan100sic               2851
IHadInt                  2806
chemthan                 2707
Akikaze                  2540
Heart_Blue               2512
2-SAD                    2499
spsmoj2                  2491
bojverdict1              2475
xjlinyueru               2475
freebsdx                 2469
Ashishgup                2428
Name: handle, dtype: int64

### Change paths to match ones on server

In [15]:
def transform_path(path):
    path = path.replace('\\', '/')
    path = path.replace('codeforces', 'codeforces-clean')
    return path

def token_path(path):
    cf, competition, problem, submissions, submission_id = path.split('/')
    return f'tokens/{competition}/{competition}.{problem}.{submission_id}&tokens.csv'

In [16]:
transform_path(index.path[0]), token_path(transform_path(index.path[0]))

('codeforces-clean/1/A/submissions/10005159.program.cpp',
 'tokens/1/1.A.10005159.program.cpp&tokens.csv')

In [17]:
index.path = index.path.map(transform_path)

/home/gb/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [21]:
index['token_path'] = index.path.map(token_path)

In [22]:
index.to_csv(get_file('index_clean.csv'), index=False)

### Load clean index

In [3]:
index = pd.read_csv(get_file('index_clean.csv'))

### Load tokens

In [97]:
def collect_tokens(ind, total_inds):
    token_counts = Counter()
    start = ind * (len(index) // total_inds)
    end = (ind + 1) * (len(index) // total_inds)
    for i, token_path in enumerate(index.token_path[start:end]):
        if i % 10000 == 0:
            print(f'Proc {ind}, {i / (end-start) * 100}%')
        submit_counts = pd.read_csv(get_file(token_path))
        for token, cnt in zip(submit_counts['token'], submit_counts['count']):
            token_counts[token] += cnt
        del submit_counts
    return token_counts

In [ ]:
n_batches = 8
pool = mp.Pool(mp.cpu_count())
batch_counts = [pool.apply_async(collect_tokens, args=(i, n_batches)) for i in range(n_batches)]
for count in batch_counts:
    count.wait()
pool.close()

In [101]:
token_counts = sum(map(lambda bc: bc.get(), batch_counts), Counter())

In [104]:
pickle.dump(token_counts, open('token_counts.pkl', 'wb'))

In [110]:
for token, cnt in token_counts.most_common()[:100]:
    print(token, cnt)

i 141270436
n 54147446
a 39652331
j 33981371
x 26124219
b 19457101
cout 19239218
s 18987105
cin 18395042
k 17898230
ans 15840150
m 15755628
v 13353585
main 12665967
y 11690562
c 11376150
ll 10579322
l 10151217
r 9571585
t 9289622
p 9283884
endl 7857174
d 6534039
scanf 6133572
size 5872110
printf 5715502
u 5653228
N 5582626
sum 5200758
cnt 5162754
vector 4922106
q 4911360
f 4828098
dp 4583430
arr 4264706
res 4207639
string 3989093
max 3597153
min 3295143
push_back 3225735
first 2957946
second 2784804
mid 2634279
g 2631555
w 2567587
A 2556143
num 2475539
pos 2459387
it 2429420
e 2343484
h 2340294
T 2315821
end 2279965
z 2220041
str 2198991
begin 2196866
sort 2161156
tmp 2117998
tie 2101672
std 2094193
cur 2092813
sync_with_stdio 2087003
temp 2067956
flag 2010742
st 1933235
maxn 1868008
len 1861923
val 1783902
pair 1708199
node 1691793
mod 1630326
mp 1610226
count 1609308
LL 1586212
dfs 1572702
vis 1499357
ch 1491296
id 1488249
mx 1469607
ret 1456086
M 1434644
s1 1410078
pb 1383384
S 1340

In [113]:
unique_tokens = len(token_counts)
total_tokens = sum(token_counts.values())

In [114]:
unique_tokens, total_tokens

(1241516, 940263220)

In [121]:
total = 0
unique = 0
ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999, 1.]
cur_ratio = 0
for token, cnt in token_counts.most_common():
    total += cnt
    unique += 1
    if total / total_tokens >= ratios[cur_ratio]:
        print(f'At {ratios[cur_ratio]}: {unique} tokens, {unique / unique_tokens:.3f}')
        cur_ratio += 1

At 0.1: 1 tokens, 0.000
At 0.2: 2 tokens, 0.000
At 0.3: 5 tokens, 0.000
At 0.4: 10 tokens, 0.000
At 0.5: 17 tokens, 0.000
At 0.6: 29 tokens, 0.000
At 0.7: 59 tokens, 0.000
At 0.8: 141 tokens, 0.000
At 0.9: 573 tokens, 0.000
At 0.99: 85182 tokens, 0.069
At 0.999: 841973 tokens, 0.678
At 1.0: 1241516 tokens, 1.000


In [124]:
total = 0
unique = 0
threshold = [100000, 10000, 1000, 100, 50, 20, 10, 5, 3, 1]
cur_th = 0
for token, cnt in token_counts.most_common():
    total += cnt
    unique += 1
    if cur_th < len(threshold) and cnt <= threshold[cur_th]:
        print(f'At {threshold[cur_th]}: {unique} tokens, {unique / unique_tokens:.3f}')
        cur_th += 1

At 100000: 515 tokens, 0.000
At 10000: 2186 tokens, 0.002
At 1000: 10508 tokens, 0.008
At 100: 55703 tokens, 0.045
At 50: 92902 tokens, 0.075
At 20: 186810 tokens, 0.150
At 10: 327058 tokens, 0.263
At 5: 587850 tokens, 0.473
At 3: 840847 tokens, 0.677
At 1: 1198276 tokens, 0.965


### Load saved token counts

In [4]:
token_counts = pickle.load(open('token_counts.pkl', 'rb'))

### Load tags

In [6]:
def path_to_problem(path):
    d, c, p, _, _ = path.split('/')
    return '/'.join((d, c, p))

In [7]:
problems = set()

for path in index.path:
    problems.add(path_to_problem(path))

In [8]:
tags = {}
for problem in problems:
    meta = json.load(open(get_file(os.path.join(problem, 'meta.json')), 'r'))
    tags[problem] = meta['tags']

In [9]:
tags

{'codeforces-clean/209/A': ['dp', 'math'],
 'codeforces-clean/1003/C': ['brute force', 'implementation', 'math'],
 'codeforces-clean/638/A': ['*special', 'constructive algorithms', 'math'],
 'codeforces-clean/489/F': ['combinatorics', 'dp'],
 'codeforces-clean/551/E': ['binary search',
  'data structures',
  'implementation'],
 'codeforces-clean/342/A': ['greedy', 'implementation'],
 'codeforces-clean/631/E': ['data structures', 'dp', 'geometry'],
 'codeforces-clean/628/E': ['data structures', 'implementation'],
 'codeforces-clean/332/B': ['data structures', 'dp', 'implementation'],
 'codeforces-clean/816/D': ['combinatorics', 'math'],
 'codeforces-clean/79/E': ['math'],
 'codeforces-clean/793/F': ['data structures', 'divide and conquer', 'dp'],
 'codeforces-clean/990/D': ['constructive algorithms',
  'graphs',
  'implementation'],
 'codeforces-clean/238/A': ['constructive algorithms', 'math'],
 'codeforces-clean/290/B': ['*special', 'implementation'],
 'codeforces-clean/75/C': ['binar

In [9]:
def collect_token_to_tag(ind, total_inds, min_token_count=50):
    start = ind * (len(index) // total_inds)
    end = (ind + 1) * (len(index) // total_inds)
    token_to_tag_counts = {}
    for i, (path, token_path) in enumerate(zip(index.path[start:end], index.token_path[start:end])):
        if i % 10000 == 0:
            print(f'Proc {ind}, {i / (end-start) * 100}%')

        problem = path_to_problem(path)
        
        submit_counts = pd.read_csv(get_file(token_path))
        for token, cnt in zip(submit_counts['token'], submit_counts['count']):
            if token_counts[token] >= min_token_count:
                if token not in token_to_tag_counts:
                    token_to_tag_counts[token] = Counter()
        
                for tag in tags[problem]:            
                    token_to_tag_counts[token][tag] += cnt

        del submit_counts
    return token_to_tag_counts

In [10]:
n_batches = 8
pool = mp.Pool(mp.cpu_count())
batch_tag_token_counts = [pool.apply_async(collect_token_to_tag, args=(i, n_batches)) for i in range(n_batches)]
for count in batch_tag_token_counts:
    count.wait()
pool.close()

Proc 4, 0.0%
Proc 6, 0.0%
Proc 1, 0.0%
Proc 5, 0.0%
Proc 0, 0.0%
Proc 3, 0.0%
Proc 2, 0.0%
Proc 7, 0.0%
Proc 4, 0.6316377229207434%
Proc 5, 0.6316377229207434%
Proc 7, 0.6316377229207434%
Proc 1, 0.6316377229207434%
Proc 6, 0.6316377229207434%
Proc 2, 0.6316377229207434%
Proc 0, 0.6316377229207434%
Proc 3, 0.6316377229207434%
Proc 1, 1.2632754458414868%
Proc 4, 1.2632754458414868%
Proc 5, 1.2632754458414868%
Proc 7, 1.2632754458414868%
Proc 6, 1.2632754458414868%
Proc 0, 1.2632754458414868%
Proc 2, 1.2632754458414868%
Proc 3, 1.2632754458414868%
Proc 4, 1.89491316876223%
Proc 5, 1.89491316876223%
Proc 1, 1.89491316876223%
Proc 6, 1.89491316876223%
Proc 0, 1.89491316876223%
Proc 7, 1.89491316876223%
Proc 2, 1.89491316876223%
Proc 3, 1.89491316876223%
Proc 0, 2.5265508916829735%
Proc 1, 2.5265508916829735%
Proc 5, 2.5265508916829735%
Proc 4, 2.5265508916829735%
Proc 6, 2.5265508916829735%
Proc 7, 2.5265508916829735%
Proc 3, 2.5265508916829735%
Proc 2, 2.5265508916829735%
Proc 0, 3.158188

Proc 0, 24.002233470988248%
Proc 1, 24.002233470988248%
Proc 3, 24.002233470988248%
Proc 4, 24.002233470988248%
Proc 2, 24.002233470988248%
Proc 5, 24.002233470988248%
Proc 6, 24.002233470988248%
Proc 7, 24.002233470988248%
Proc 0, 24.63387119390899%
Proc 1, 24.63387119390899%
Proc 4, 24.63387119390899%
Proc 3, 24.63387119390899%
Proc 2, 24.63387119390899%
Proc 5, 24.63387119390899%
Proc 6, 24.63387119390899%
Proc 7, 24.63387119390899%
Proc 0, 25.26550891682973%
Proc 1, 25.26550891682973%
Proc 4, 25.26550891682973%
Proc 3, 25.26550891682973%
Proc 5, 25.26550891682973%
Proc 2, 25.26550891682973%
Proc 6, 25.26550891682973%
Proc 7, 25.26550891682973%
Proc 0, 25.897146639750478%
Proc 1, 25.897146639750478%
Proc 4, 25.897146639750478%
Proc 3, 25.897146639750478%
Proc 2, 25.897146639750478%
Proc 5, 25.897146639750478%
Proc 6, 25.897146639750478%
Proc 7, 25.897146639750478%
Proc 0, 26.52878436267122%
Proc 1, 26.52878436267122%
Proc 4, 26.52878436267122%
Proc 3, 26.52878436267122%
Proc 2, 26.5

Proc 6, 47.37282921905575%
Proc 7, 47.37282921905575%
Proc 0, 48.004466941976496%
Proc 1, 48.004466941976496%
Proc 3, 48.004466941976496%
Proc 4, 48.004466941976496%
Proc 2, 48.004466941976496%
Proc 5, 48.004466941976496%
Proc 6, 48.004466941976496%
Proc 7, 48.004466941976496%
Proc 0, 48.63610466489724%
Proc 1, 48.63610466489724%
Proc 3, 48.63610466489724%
Proc 2, 48.63610466489724%
Proc 4, 48.63610466489724%
Proc 5, 48.63610466489724%
Proc 6, 48.63610466489724%
Proc 7, 48.63610466489724%
Proc 0, 49.26774238781798%
Proc 1, 49.26774238781798%
Proc 3, 49.26774238781798%
Proc 2, 49.26774238781798%
Proc 4, 49.26774238781798%
Proc 5, 49.26774238781798%
Proc 6, 49.26774238781798%
Proc 7, 49.26774238781798%
Proc 0, 49.899380110738726%
Proc 1, 49.899380110738726%
Proc 3, 49.899380110738726%
Proc 2, 49.899380110738726%
Proc 4, 49.899380110738726%
Proc 5, 49.899380110738726%
Proc 6, 49.899380110738726%
Proc 7, 49.899380110738726%
Proc 0, 50.53101783365946%
Proc 1, 50.53101783365946%
Proc 3, 50.5

Proc 7, 71.375062690044%
Proc 6, 71.375062690044%
Proc 1, 72.00670041296475%
Proc 0, 72.00670041296475%
Proc 3, 72.00670041296475%
Proc 2, 72.00670041296475%
Proc 5, 72.00670041296475%
Proc 4, 72.00670041296475%
Proc 7, 72.00670041296475%
Proc 6, 72.00670041296475%
Proc 1, 72.63833813588548%
Proc 0, 72.63833813588548%
Proc 3, 72.63833813588548%
Proc 2, 72.63833813588548%
Proc 5, 72.63833813588548%
Proc 4, 72.63833813588548%
Proc 7, 72.63833813588548%
Proc 6, 72.63833813588548%
Proc 1, 73.26997585880622%
Proc 0, 73.26997585880622%
Proc 3, 73.26997585880622%
Proc 2, 73.26997585880622%
Proc 5, 73.26997585880622%
Proc 4, 73.26997585880622%
Proc 7, 73.26997585880622%
Proc 1, 73.90161358172698%
Proc 6, 73.26997585880622%
Proc 0, 73.90161358172698%
Proc 3, 73.90161358172698%
Proc 2, 73.90161358172698%
Proc 5, 73.90161358172698%
Proc 4, 73.90161358172698%
Proc 7, 73.90161358172698%
Proc 6, 73.90161358172698%
Proc 1, 74.53325130464772%
Proc 0, 74.53325130464772%
Proc 3, 74.53325130464772%
Proc 

Proc 7, 95.37729616103225%
Proc 6, 95.37729616103225%
Proc 3, 96.00893388395299%
Proc 2, 96.00893388395299%
Proc 5, 96.00893388395299%
Proc 4, 96.00893388395299%
Proc 0, 96.64057160687373%
Proc 1, 96.64057160687373%
Proc 7, 96.00893388395299%
Proc 6, 96.00893388395299%
Proc 3, 96.64057160687373%
Proc 2, 96.64057160687373%
Proc 5, 96.64057160687373%
Proc 4, 96.64057160687373%
Proc 0, 97.27220932979448%
Proc 1, 97.27220932979448%
Proc 7, 96.64057160687373%
Proc 6, 96.64057160687373%
Proc 3, 97.27220932979448%
Proc 2, 97.27220932979448%
Proc 5, 97.27220932979448%
Proc 4, 97.27220932979448%
Proc 0, 97.90384705271522%
Proc 1, 97.90384705271522%
Proc 7, 97.27220932979448%
Proc 6, 97.27220932979448%
Proc 3, 97.90384705271522%
Proc 2, 97.90384705271522%
Proc 5, 97.90384705271522%
Proc 4, 97.90384705271522%
Proc 0, 98.53548477563596%
Proc 1, 98.53548477563596%
Proc 7, 97.90384705271522%
Proc 6, 97.90384705271522%
Proc 3, 98.53548477563596%
Proc 2, 98.53548477563596%
Proc 5, 98.53548477563596%
P

In [12]:
token_tag_counts = {}
for token_tag in batch_tag_token_counts:
    token_tag = token_tag.get()
    for token, tag_counts in token_tag.items():
        if token not in token_tag_counts:
            token_tag_counts[token] = Counter()
        token_tag_counts[token] += tag_counts

In [13]:
pickle.dump(token_tag_counts, open('token_tag_counts.pkl', 'wb'))

### Load token-tag dictionary

In [5]:
token_tag_counts = pickle.load(open('token_tag_counts.pkl', 'rb'))

In [97]:
token_tag_ratios = {}
for token, tag_counts in token_tag_counts.items():
    token_tag_ratios[token] = []
    for tag, cnt in tag_counts.most_common():
        token_tag_ratios[token].append((tag, cnt / token_counts[token]))

In [98]:
def tiktok(token):
    return token_counts[token], token_tag_ratios[token]

In [106]:
for token, counts in token_tag_ratios.items():
    if token_counts[token] > 50000 and 0.7 > counts[0][1] > 0.6:
        print(token, token_counts[token], counts[:5])
        print()

ceil 127976 [('math', 0.6681955991748453), ('implementation', 0.3168094017628305), ('greedy', 0.1964118272176033), ('constructive algorithms', 0.10609801837844596), ('binary search', 0.09307213852597362)]

counter 285978 [('implementation', 0.6205966892558169), ('greedy', 0.2510752575372931), ('brute force', 0.2131667470924337), ('math', 0.1563581813985691), ('strings', 0.11811398079572555)]

string 3989093 [('implementation', 0.6008503687429699), ('strings', 0.3973640123205952), ('greedy', 0.21844940692031997), ('brute force', 0.1926884131305036), ('dp', 0.11124634096021326)]

str 2198991 [('implementation', 0.6041375339871786), ('strings', 0.4083286379980637), ('greedy', 0.23521878898094625), ('brute force', 0.18894984108620727), ('dp', 0.12054801497595943)]

count2 54130 [('implementation', 0.6404027341585073), ('greedy', 0.33026048401995195), ('math', 0.20175503417698135), ('brute force', 0.17986329207463514), ('strings', 0.17814520598559025)]

fact 164358 [('combinatorics', 0.6302

In [49]:
len(token_tag_counts)

94325

In [114]:
tiktok('remainder')

(9517,
 [('implementation', 0.47052642639487235),
  ('math', 0.45518545760218554),
  ('greedy', 0.23631396448460648),
  ('brute force', 0.1784175685615215),
  ('dp', 0.1443732268572029),
  ('number theory', 0.13974992119365345),
  ('constructive algorithms', 0.12367342649994746),
  ('binary search', 0.07302721445833771),
  ('strings', 0.07292213932962067),
  ('sortings', 0.0527477146159504),
  ('combinatorics', 0.05201218871493118),
  ('data structures', 0.041084375328359776),
  ('bitmasks', 0.03141746348639277),
  ('fft', 0.030366712199222445),
  ('dfs and similar', 0.029315960912052116),
  ('graphs', 0.02511295576337081),
  ('two pointers', 0.018598297782914785),
  ('trees', 0.016181569822423032),
  ('geometry', 0.01565619417883787),
  ('shortest paths', 0.013869916990648313),
  ('*special', 0.007985709782494483),
  ('dsu', 0.00588420720815383),
  ('matrices', 0.005568981822002732),
  ('chinese remainder theorem', 0.004833455920983503),
  ('string suffix structures', 0.00441315540611

### There are some tokens that contain valuable information about the tags